In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
import multiprocessing

In [2]:
df_2024 = pd.read_parquet('Futures_2024.parquet', engine='pyarrow')
df_2023 = pd.read_parquet('Futures_2023.parquet', engine='pyarrow')

In [3]:
df = pd.concat([df_2023, df_2024])
df = df[[col for col in df.columns if 'Close' in col]]
df.index = pd.to_datetime(df.index)
df.replace(0, pd.NA, inplace=True)
df.ffill(inplace=True)
df_resample = df.resample('D', origin ='2023-01-02 09:16:00').first()
df_resample_dropna = df_resample.dropna(how='all')

C:\Users\sambu\AppData\Local\Temp\ipykernel_26220\3307338745.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.ffill(inplace=True)


In [4]:
df_nifty= pd.read_parquet('Idx_fut.parquet', engine='pyarrow')
df_nifty = df_nifty[[col for col in df_nifty if 'NIFTY_Close' in col]]
df_nifty.index = pd.to_datetime(df_nifty.index)
df_nifty = df_nifty[df_nifty.index >= '2023-01-01 09:16:00']
nifty_frame = df_nifty.resample('D',origin='2023-01-02 09:16:00').first().dropna()
nifty_frame.drop(columns = 'BANKNIFTY_Close', inplace=True)
nifty_frame = nifty_frame[nifty_frame.index <'2024-07-02 15:30:00']

In [5]:
rel_prices = df_resample_dropna.div(nifty_frame['NIFTY_Close'], axis=0)

In [6]:
nifty_50 = ["MARUTI_Close","JSWSTEEL_Close","HDFCLIFE_Close","ASIANPAINT_Close","NTPC_Close","SBILIFE_Close","HINDALCO_Close","ADANIPORTS_Close","BHARTIARTL_Close","HEROMOTOCO_Close","KOTAKBANK_Close","ADANIENT_Close","BAJAJ-AUTO_Close","ITC_Close","CIPLA_Close","SUNPHARMA_Close","TECHM_Close","ONGC_Close","TATASTEEL_Close","BPCL_Close","LT_Close","ULTRACEMCO_Close","SHRIRAMFIN_Close","HCLTECH_Close","TCS_Close","COALINDIA_Close","HINDUNILVR_Close","EICHERMOT_Close","ICICIBANK_Close","DIVISLAB_Close","BAJAJFINSV_Close","SBIN_Close","NESTLEIND_Close","HDFCBANK_Close","WIPRO_Close","TITAN_Close","INDUSINDBK_Close","LTIM_Close","POWERGRID_Close","AXISBANK_Close","BAJFINANCE_Close","TATAMOTORS_Close","INFY_Close","M&M_Close","GRASIM_Close","RELIANCE_Close","TATACONSUM_Close","APOLLOHOSP_Close","DRREDDY_Close","BRITANNIA_Close"]

stock_prices = df_resample_dropna[nifty_50]
relative_prices = rel_prices[nifty_50]
full_price_data = df[nifty_50]

adjusted_index = full_price_data.index - pd.Timedelta(seconds=59)
full_price_data.index = adjusted_index
full_price_data = full_price_data[full_price_data.index >= '2023-01-16 09:14:00']

In [7]:
full_price_data

,MARUTI_Close,JSWSTEEL_Close,HDFCLIFE_Close,ASIANPAINT_Close,NTPC_Close,SBILIFE_Close,HINDALCO_Close,ADANIPORTS_Close,BHARTIARTL_Close,HEROMOTOCO_Close,...,BAJFINANCE_Close,TATAMOTORS_Close,INFY_Close,M&M_Close,GRASIM_Close,RELIANCE_Close,TATACONSUM_Close,APOLLOHOSP_Close,DRREDDY_Close,BRITANNIA_Close
DateTime,,,,,,,,,,,,,,,,,,,,,
2023-01-16 09:15:00,8532.95,770.00,611.00,2926.00,168.80,1319.00,492.65,798.20,769.60,2746.30,...,6015.00,415.00,1517.35,1331.70,1667.05,2485.25,756.05,4343.25,4348.20,4338.30
2023-01-16 09:16:00,8518.00,766.00,612.80,2931.05,169.00,1321.95,490.65,795.25,769.15,2740.65,...,6025.95,413.65,1515.40,1330.20,1669.00,2483.95,755.70,4366.15,4340.40,4337.35
2023-01-16 09:17:00,8516.50,765.00,611.05,2935.35,168.60,1320.45,491.30,795.55,768.65,2739.90,...,6055.00,413.50,1511.70,1326.95,1670.25,2485.80,754.80,4358.75,4337.00,4341.35
2023-01-16 09:18:00,8522.95,766.05,612.25,2937.00,168.00,1319.85,491.00,795.65,769.00,2743.65,...,6051.00,413.00,1514.00,1326.15,1668.80,2487.00,754.80,4373.25,4352.00,4343.00
2023-01-16 09:19:00,8517.70,764.05,612.35,2935.45,167.85,1318.90,490.35,795.05,770.15,2744.05,...,6033.10,412.80,1513.20,1326.80,1669.20,2485.75,754.75,4373.00,4347.50,4338.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-02 15:26:00,12102.80,936.90,592.90,2940.00,372.10,1500.20,698.65,1482.15,1426.25,5589.30,...,7186.10,985.40,1621.95,2860.35,2754.55,3142.60,1112.35,6127.25,6362.15,5441.20
2024-07-02 15:27:00,12102.05,937.30,593.30,2939.45,372.10,1500.55,698.95,1483.50,1427.60,5585.05,...,7197.45,985.60,1622.60,2859.00,2755.95,3142.70,1113.75,6128.75,6365.20,5442.45
2024-07-02 15:28:00,12096.00,937.40,593.00,2937.25,372.15,1498.95,699.55,1482.90,1427.65,5581.85,...,7199.00,985.90,1620.50,2861.55,2754.35,3143.10,1113.00,6127.95,6362.85,5439.75


In [8]:
multiplier = pd.read_csv('instrument_list.csv')
multiplier = multiplier.drop(columns=[col for col in multiplier.columns if col not in ['name', 'segment', 'lot_size']])
multiplier_nfo = multiplier[multiplier['segment'] == 'NFO-FUT']
filtered_df = multiplier_nfo.drop_duplicates(subset='name', keep='first')
valid_stocks = filtered_df['name'].unique()
valid_stock_columns = [f"{stock}_Close" for stock in valid_stocks]
columns_to_keep = [col for col in valid_stock_columns if col in stock_prices.columns]
filtered_price_df = stock_prices[columns_to_keep]
filtered_stock_names = [col.split('_')[0] for col in filtered_price_df.columns]
filtered_multiplier_df = filtered_df[filtered_df['name'].isin(filtered_stock_names)]
filtered_multiplier_df.drop(columns = 'segment', inplace=True)
filtered_multiplier_df.set_index('name', inplace=True)
filtered_multiplier_df.columns = ['mult']
mult_list = filtered_multiplier_df
mult_list.index = mult_list.index + '_Close'

C:\Users\sambu\AppData\Local\Temp\ipykernel_26220\984550555.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_multiplier_df.drop(columns = 'segment', inplace=True)


In [13]:
class PortfolioMomentumStrategy:
    def __init__(self, stock_prices, relative_prices, full_price_data, mult, lookback_period, num_stocks, max_positions, initial_capital=200000000):
        self.stock_prices = stock_prices
        self.relative_prices = relative_prices
        self.full_price_data = full_price_data
        self.mult = mult
        self.max_positions = max_positions
        self.lookback_period = lookback_period
        self.num_stocks = int(min(max_positions // 2, num_stocks))
        self.available_capital = initial_capital
        self.current_positions = {stock: 0 for stock in stock_prices.columns}
        self.pnl_series = pd.Series(index = stock_prices.index)
        self.trade_log = []
        self.exit_log = []
        self.transaction_costs_rate = 0.0011
        self.position_matrix = pd.DataFrame(index=stock_prices.index, columns=stock_prices.columns)
        #self.position_df = pd.DataFrame(index = full_price_data, columns = stock_prices.columns)
        

    def calculate_momentum(self, data, current_time):
        lookback_time = current_time - pd.Timedelta(days=self.lookback_period)
        if lookback_time in data.index:
            return (data.loc[current_time]/ data.loc[lookback_time] - 1)  # Returns the momentum between now and that time
        else:
            return pd.Series(np.nan, index=data.columns) #In cases of large lookback period
    
    def rank_stocks(self, current_time):
        momentum_scores = self.calculate_momentum(self.relative_prices, current_time)
        ranked_stocks = momentum_scores.sort_values(ascending = False) #Ranks stocks top to bottom of momentum in the time period
        return ranked_stocks
    
    def allocate_positions(self, ranked_stocks, current_prices):
        top_stocks = ranked_stocks.head(self.num_stocks)
        bottom_stocks = ranked_stocks.tail(self.num_stocks)


        long_value = self.available_capital / 2
        short_value = self.available_capital / 2 #Assign half capital to long, half to short

        max_exp_per_stock = 10000000

        long_positions = {}  #To track which stocks we go long and short on and the position size
        short_positions = {}

        #So we don't repeatedly buy/sell if a stock remains in the top/bottom n stocks
        top_stocks = [stock for stock in top_stocks.index if self.current_positions[stock] == 0]
        bottom_stocks = [stock for stock in bottom_stocks.index if self.current_positions[stock] == 0]

        #Tracks total positions, i.e number of stocks in our portfolio
        total_positions = sum(1 for pos in self.current_positions.values() if pos != 0)

        #Go long with the top n
        for stock in top_stocks:
            if total_positions < self.max_positions:  #Only if we don't go over max_positions stocks
                
                #Position size will be the minimum of the exposure per stock/ (total price of contract) and available capital / (total price of contract)
                
                position_size = int(min(max_exp_per_stock / (current_prices[stock] * self.mult.loc[stock].values), long_value / (current_prices[stock] * self.mult.loc[stock].values)))
                position_cost = position_size * current_prices[stock] * self.mult.loc[stock].values
                
                #Only carry out trade if we have enough capital
                
                if position_cost <= long_value:
                    long_positions[stock] = position_size
                    long_value -= position_cost
                    self.trade_log.append({'date': current_prices.name, 'stock': stock, 'position': 'long', 'size': position_size, 'cost': position_cost[0], 'trading price' : current_prices[stock], 'trade_type' : 'entry', 'multiplier': self.mult.loc[stock].values[0]})
                    total_positions += 1

        #Same process, but for short.
        for stock in bottom_stocks:
            if total_positions < self.max_positions:
                position_size = int(min(max_exp_per_stock / (current_prices[stock] * self.mult.loc[stock].values), short_value / (current_prices[stock] * self.mult.loc[stock].values)))
                position_cost = position_size * current_prices[stock] * self.mult.loc[stock].values
                if position_cost <= short_value:
                    short_positions[stock] = position_size
                    short_value -= position_cost
                    self.trade_log.append({'date': current_prices.name, 'stock': stock, 'position': 'short', 'size': position_size, 'cost': position_cost[0], 'trading price': current_prices[stock], 'trade_type' : 'entry', 'multiplier': self.mult.loc[stock].values[0]})
                    total_positions += 1

        return long_positions, short_positions  


    def update_positions(self, long_positions, short_positions, previous_prices, current_prices):

        liquidated_pnl = 0
        total_positions = sum(1 for pos in self.current_positions.values() if pos != 0)

        #If stocks move out of the top and bottom n, the position is closed and we calculate the pnl of the close

        for stock in self.current_positions:
            if stock not in long_positions and stock not in short_positions:
                if self.current_positions[stock] != 0:
                    liquidated_pnl = self.current_positions[stock] * (current_prices[stock]) * self.mult.loc[stock].values
                    total_positions -= 1
                    self.exit_log.append({'date': current_prices.name, 'stock': stock, 'position': ' Closing long' if self.current_positions[stock] > 0 else ' Closing short',
                                        'size': abs(self.current_positions[stock]),
                                        'cost': abs(self.current_positions[stock]) * previous_prices[stock] * self.mult.loc[stock].values[0],
                                        'trading price': current_prices[stock],
                                        'trade_type': 'exit', 'multiplier': self.mult.loc[stock].values[0]})
                    self.current_positions[stock] = 0

        #Current_positions tracks the stocks in our portfolio and the position size
        
        for stock, position in long_positions.items():
            self.current_positions[stock] = position

        for stock, position in short_positions.items():
            self.current_positions[stock] = -position

        return liquidated_pnl
    
    def update_position_matrix(self, current_time):
        current_prices = self.stock_prices.loc[current_time]
        for stock, position in self.current_positions.items():
            self.position_matrix.at[current_time, stock] = position
  
    #Simple function to track pnl

    def calculate_pnl(self, long_positions, short_positions, previous_prices, current_prices):
        pnl = 0

        for stock, position in self.current_positions.items():
            price_change = current_prices[stock] - previous_prices[stock]
            pnl += position * price_change * self.mult.loc[stock].values + self.update_positions(long_positions, short_positions, previous_prices, current_prices)
        return pnl
    
    
    
    def calculate_transaction_costs(self, trades):
        total_transaction_costs = 0
        for trade in trades:
            turnover = trade['size'] * trade['trading price']
            total_transaction_costs += turnover * self.transaction_costs_rate
        return total_transaction_costs

    def trade(self):

        #Indexing to be able to select correct prices

        unique_dates=self.stock_prices.index
        unique_dates=unique_dates[self.lookback_period:]

        next_unique_dates=self.stock_prices.index
        next_unique_dates=next_unique_dates[self.lookback_period-1:]

        #For every day, apply the strategy
        for current_time,prev_time in zip(unique_dates,next_unique_dates):
            previous_prices = self.stock_prices.loc[prev_time]
            current_prices = self.stock_prices.loc[current_time]
            ranked_stocks = self.rank_stocks(current_time)
            long_positions, short_positions = self.allocate_positions(ranked_stocks, current_prices)
            self.update_positions(long_positions, short_positions, previous_prices, current_prices)
            pnl_raw = self.calculate_pnl(long_positions, short_positions, previous_prices, current_prices) 
            liquidated_pnl = self.update_positions(long_positions, short_positions, previous_prices, current_prices)
            pnl = pnl_raw + liquidated_pnl
            self.update_position_matrix(current_time)

            day_trades = [trade for trade in self.trade_log if trade['date'] == current_time]
            if day_trades:  
                transaction_costs = self.calculate_transaction_costs(day_trades)
                pnl -= transaction_costs
            
            self.pnl_series[current_time] = pnl 

            self.available_capital += pnl

        return pd.Series(self.pnl_series, index=self.stock_prices.index[self.lookback_period:]).cumsum(), self.position_matrix
    

    



In [12]:
strategy = PortfolioMomentumStrategy(stock_prices, full_price_data, relative_prices, mult_list, 10, 5, 20)
pnl_series, position_matrix = strategy.trade()
#position_matrix.dropna(inplace=True)

C:\Users\sambu\AppData\Local\Temp\ipykernel_26220\133527943.py:58: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  position_size = int(min(max_exp_per_stock / (current_prices[stock] * self.mult.loc[stock].values), long_value / (current_prices[stock] * self.mult.loc[stock].values)))
C:\Users\sambu\AppData\Local\Temp\ipykernel_26220\133527943.py:72: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  position_size = int(min(max_exp_per_stock / (current_prices[stock] * self.mult.loc[stock].values), short_value / (current_prices[stock] * self.mult.loc[stock].values)))


KeyError: Timestamp('2023-01-26 09:16:00')

In [ ]:
price_change = full_price_data.diff()
stock_prices_daily = stock_prices[stock_prices.index >= '2023-01-16 09:15:00']
minutely_position_matrix = pd.DataFrame(index = full_price_data.index, columns = position_matrix.columns)
for column in position_matrix.columns:
    minutely_position_matrix[column] = minutely_position_matrix[column].fillna(method='ffill')

minutely_position_matrix.update(position_matrix.reindex(minutely_position_matrix.index, method='ffill'))

minutely_price_position = minutely_position_matrix.mul(price_change)
minutely_price_position.dropna(inplace=True)

mult_df = mult_list.T
mult_df = mult_df.reindex(columns=minutely_price_position.columns)

minutely_stock_pnl = minutely_price_position * mult_df.values

minutely_pnl = minutely_stock_pnl.sum(axis=1)

initial_cap = 200000000
cumulative_minutely_pnl = minutely_pnl.cumsum()
rel_cumulative_minutely_pnl = cumulative_minutely_pnl / initial_cap

In [ ]:
rel_cumulative_minutely_pnl.to_csv('minute_pnl_script1_test.csv')